In [1]:
import pandas as pd
import numpy as np
import pymongo
import json

from config import DBConfig

In [2]:
# 登录 MongoDB
myclient = pymongo.MongoClient(DBConfig.ip, username=DBConfig.username, 
                               password=DBConfig.password)
# 显示数据库列表
print(myclient.list_database_names())

['QTA_2020', 'admin', 'config', 'local']


In [3]:
DBConfig.db_name

'QTA_2020'

In [4]:
# 连接数据库
mydb = myclient[DBConfig.db_name]

# 显示collection列表
print(mydb.list_collection_names())

['daily_factor', 'minute_quote', 'index_weight', 'daily_quote', 'stock_info', 'daily_style_factor']


In [5]:
# 创建demo数据
l = [pd.date_range(start='2020-10-10', end='2020-10-15'),[str(x+1).zfill(6)+'.sz' for x in range(6)],
     np.random.rand(6), np.random.rand(6), np.random.rand(6)]
daily_factor_df = pd.DataFrame(l).T
daily_factor_df.columns=['datetime', 'code', 'factor1', 'factor2', 'factor3']
daily_factor_df

,datetime,code,factor1,factor2,factor3
0,2020-10-10,000001.sz,0.885987,0.131043,0.0161896
1,2020-10-11,000002.sz,0.576781,0.453938,0.344057
2,2020-10-12,000003.sz,0.206857,0.115144,0.491489
3,2020-10-13,000004.sz,0.291992,0.431848,0.722746
4,2020-10-14,000005.sz,0.809319,0.15864,0.291753
5,2020-10-15,000006.sz,0.505646,0.892285,0.723272


In [6]:
# datetime列统一用python里的datetime类型或timestamp类型
daily_factor_df['datetime'].iat[0]

Timestamp('2020-10-10 00:00:00')

## 增

In [7]:
def insert_dataframe(df, collection):
    collection.insert_many(df.to_dict('record'))

In [8]:
daily_factor_collection = mydb['daily_factor']
# 把dataframe插入数据库
insert_dataframe(daily_factor_df, daily_factor_collection)

## 查

In [9]:
cursor = daily_factor_collection.find({'code': '000001.sz'})
list(cursor)

[{'_id': ObjectId('5f8d8a7be3b12354117e8278'),
  'datetime': datetime.datetime(2020, 10, 10, 0, 0),
  'code': '000001.sz',
  'factor1': 0.8859871686529796,
  'factor2': 0.13104332015808018,
  'factor3': 0.01618958512237101}]

In [10]:
# 查询特定日期区间的数据
cursor = daily_factor_collection.find({'datetime':{'$gte': pd.to_datetime('2020-10-11'),
                                                  '$lt': pd.to_datetime('2020-10-14')}
                                      })
l = list(cursor)
l

[{'_id': ObjectId('5f8d8a7be3b12354117e8279'),
  'datetime': datetime.datetime(2020, 10, 11, 0, 0),
  'code': '000002.sz',
  'factor1': 0.5767808384028988,
  'factor2': 0.453938366319255,
  'factor3': 0.3440574423407401},
 {'_id': ObjectId('5f8d8a7be3b12354117e827a'),
  'datetime': datetime.datetime(2020, 10, 12, 0, 0),
  'code': '000003.sz',
  'factor1': 0.20685715879259303,
  'factor2': 0.11514358284960324,
  'factor3': 0.4914894191778306},
 {'_id': ObjectId('5f8d8a7be3b12354117e827b'),
  'datetime': datetime.datetime(2020, 10, 13, 0, 0),
  'code': '000004.sz',
  'factor1': 0.2919922422886886,
  'factor2': 0.4318484829240453,
  'factor3': 0.7227461454686258}]

In [11]:
# 查找特定的几只股票的数据
cursor = daily_factor_collection.find({'code': {'$in': ['000001.sz', '000002.sz', '000003.sz']
                                               }
                                      })
l = list(cursor)
df = pd.DataFrame(l)
df

,_id,datetime,code,factor1,factor2,factor3
0,5f8d5c2ba253674a09707868,2020-10-10,000001.sz,0.082828,0.293359,0.276362
1,5f8d5c2ba253674a09707869,2020-10-11,000002.sz,0.537638,0.956017,0.301836
2,5f8d5c2ba253674a0970786a,2020-10-12,000003.sz,0.059053,0.414332,0.936546


In [12]:
# 查找除了特定股票之外的所有数据
cursor = daily_factor_collection.find({'code': {'$nin': ['000001.sz']
                                               }
                                      })
l = list(cursor)
df = pd.DataFrame(l)
df

,_id,datetime,code,factor1,factor2,factor3
0,5f8d5c2ba253674a09707869,2020-10-11,000002.sz,0.537638,0.956017,0.301836
1,5f8d5c2ba253674a0970786a,2020-10-12,000003.sz,0.059053,0.414332,0.936546
2,5f8d5c2ba253674a0970786b,2020-10-13,000004.sz,0.587633,0.803884,0.402281
3,5f8d5c2ba253674a0970786c,2020-10-14,000005.sz,0.261762,0.910283,0.149412
4,5f8d5c2ba253674a0970786d,2020-10-15,000006.sz,0.859574,0.293090,0.087748


In [11]:
# 关于封装接口
class DataBase:
    def __init__(self, config):
        self.config = config
        # login
        self.client = pymongo.MongoClient(config.ip, username=config.username, 
                               password=config.password)
        self.db = self.client[config.db_name]
        
    def get_daily_factor(self, code_list, factor_list, start_date, end_date):
        start_date, end_date = pd.to_datetime(start_date), pd.to_datetime(end_date)
        collection = self.db['daily_factor']
        
        find_d = {'code': {'$in': code_list},
            'datetime': {
                '$gte': start_date,
                '$lte': end_date},
            }
        to_show_d = {'_id': 1, 'datetime': 1, 'code':1}
        for factor in factor_list:
            to_show_d[factor] = 1
            
        cursor = collection.find(find_d, to_show_d)
        l = list(cursor)
        if len(l) > 0:
            return pd.DataFrame(l)
        else:
            return None
        
    def get_daily_quote(self, code_list, start_date, end_date):
        pass
    
    def write_daily_factor(self, df):
        pass
    
    # ......

In [14]:
db = DataBase(DBConfig)
df = db.get_daily_factor(code_list=['000001.sz', '000002.sz', '000003.sz'],
                   factor_list=['factor1',],
                   start_date='2020-10-11',
                   end_date=pd.to_datetime('2020-10-14'))
df

,_id,datetime,code,factor1
0,5f8d5c2ba253674a09707869,2020-10-11,000002.sz,0.537638
1,5f8d5c2ba253674a0970786a,2020-10-12,000003.sz,0.059053


## 改

In [15]:
# 改之前必须先查出来
cursor = daily_factor_collection.find({})  # 不输入查找条件即全查询
l = list(cursor)
df = pd.DataFrame(l)
df

,_id,datetime,code,factor1,factor2,factor3
0,5f8d5c2ba253674a09707868,2020-10-10,000001.sz,0.082828,0.293359,0.276362
1,5f8d5c2ba253674a09707869,2020-10-11,000002.sz,0.537638,0.956017,0.301836
2,5f8d5c2ba253674a0970786a,2020-10-12,000003.sz,0.059053,0.414332,0.936546
3,5f8d5c2ba253674a0970786b,2020-10-13,000004.sz,0.587633,0.803884,0.402281
4,5f8d5c2ba253674a0970786c,2020-10-14,000005.sz,0.261762,0.910283,0.149412
5,5f8d5c2ba253674a0970786d,2020-10-15,000006.sz,0.859574,0.293090,0.087748


In [16]:
# 想象这样一个场景，我们给每个股票每天的数据算好了新的因子值
df = pd.DataFrame(l)
df['new_factor'] = np.random.rand(len(df))
df

,_id,datetime,code,factor1,factor2,factor3,new_factor
0,5f8d5c2ba253674a09707868,2020-10-10,000001.sz,0.082828,0.293359,0.276362,0.903263
1,5f8d5c2ba253674a09707869,2020-10-11,000002.sz,0.537638,0.956017,0.301836,0.964592
2,5f8d5c2ba253674a0970786a,2020-10-12,000003.sz,0.059053,0.414332,0.936546,0.235137
3,5f8d5c2ba253674a0970786b,2020-10-13,000004.sz,0.587633,0.803884,0.402281,0.138379
4,5f8d5c2ba253674a0970786c,2020-10-14,000005.sz,0.261762,0.910283,0.149412,0.588856
5,5f8d5c2ba253674a0970786d,2020-10-15,000006.sz,0.859574,0.293090,0.087748,0.765552


In [17]:
# 根据 _id 来update数据库
record_list = df[['_id', 'new_factor']].to_dict('record')
record_list

[{'_id': ObjectId('5f8d5c2ba253674a09707868'),
  'new_factor': 0.9032629202306431},
 {'_id': ObjectId('5f8d5c2ba253674a09707869'),
  'new_factor': 0.9645917330910613},
 {'_id': ObjectId('5f8d5c2ba253674a0970786a'),
  'new_factor': 0.23513656581282139},
 {'_id': ObjectId('5f8d5c2ba253674a0970786b'),
  'new_factor': 0.13837865548971018},
 {'_id': ObjectId('5f8d5c2ba253674a0970786c'),
  'new_factor': 0.588855811958797},
 {'_id': ObjectId('5f8d5c2ba253674a0970786d'),
  'new_factor': 0.7655523639089253}]

In [18]:
# 逐条更新
for record in record_list:
    daily_factor_collection.update_one({'_id': record['_id']},
                                       {'$set': {
                                           'new_factor': record['new_factor']
                                       }
                                       })

## 删

### 删一个因子

In [19]:
# 逐条更新
for record in record_list:
    daily_factor_collection.update_one({'_id': record['_id']},
                                       {'$unset': {
                                           'new_factor': ""
                                       }
                                       })

### 删除特定条件的数据

In [20]:
daily_factor_collection.delete_many({'code': '000002.sz'})

### 删除整个表（慎用！！！！！）

In [21]:
daily_factor_collection.delete_many({})